* Se puden mejorar las mascaras haciendolas de un bit

In [1]:
# Image processing
import cv2
import numpy as np
import os
import glob
from ipynb.fs.full.Utils import *

# Variables de configuración

In [2]:
segments_amount = 20 #Cantidad de segmentos
window_ratio = 0.05 # 5% con 1024 son ventanas de 51 píxeles (ojo porque quedan pixeles a la derecha si procesar)
top_crop_ratio = 0.1
width_crop_ratio = 0.1

In [3]:
kernel_ones_3 = np.ones((3,3),np.uint8)
kernel_e1 = np.array(([0,0,0,0],[0,1,1,0],[0,1,1,0],[0,0,0,0]), np.uint8)
kernel_e2 = np.array(([0,1,0],[0,1,0],[0,1,0]),np.uint8)
kernel_e3 = np.array(([0,0,0],[1,1,1],[0,0,0]),np.uint8)
kernel_e4 = np.array(([0,0,0,0,0],[0,1,1,1,0],[0,1,1,1,0],[0,1,1,1,0],[0,0,0,0,0]),np.uint8)
kernel_cruz = np.array(([0,1,0],[1,1,1],[0,1,0]),np.uint8)
kernel_x = np.array(([1,0,1],[0,1,0],[1,0,1]),np.uint8)

### Detección de lineas de cultivo por Hough
Detecta una única linea con el contenido de cultivo superior, es donde más se juntan las plantas y hay mayor densidad de puntos

In [8]:
original = take_random_picture('img/row_test_pcam/random/', 'test')
to_detect = cv2.cvtColor(original, cv2.COLOR_RGB2HSV)

In [10]:
by_color_index = img_to_color_index(original, 'cive')
print(by_color_index)
g_to_detect = segment_by_color(to_detect, binary=False)
g_to_detect = cv2.erode(g_to_detect, kernel_e1,iterations=1)
# mostrar_imgs(['original', 'seg'], [original_td, g_to_detect])

[[18.30006901 17.84511679 18.73248186 ... 18.78150266 18.80273207
  19.04844999]
 [18.15467203 15.74764868 18.72415125 ... 18.77687603 18.79263065
  19.04844999]
 [ 2.40445    18.71071447 18.70918872 ... 18.80650917 18.77849316
  19.04844999]
 ...
 [18.73709001 18.76718784 18.75004186 ... 18.93866043 19.10001767
  19.04844999]
 [18.73998274 18.76808634 18.78189639 ... 19.35411679 22.23511716
  19.04844999]
 [19.04844999 19.04844999 19.04844999 ... 19.04844999 19.04844999
  19.04844999]]


In [ ]:
# Bordes por Canny
# edges = cv2.Canny(seg_and,50,150)
# mostrar_img('edges canny', edges)

# Bordes horizontales por convolución (sabiendo que vemos las columnas cultivo de frente)
sz_v = 0.4
kernel_edge = np.array([[-1,2,-1], #Segmentos verticales
           [-1,2,-1],
           [-1,2,-1]])
edges = cv2.filter2D(g_to_detect,-1,kernel_edge)
# edges = cv2.resize(edges, ((int)(edges.shape[1]*sz_v),(int)(edges.shape[0]*sz_v)))
mostrar_img('edges conv', edges)

In [ ]:
#Realizando erociones iterativas con kernel vertical
g_to_detect_2 = cv2.erode(g_to_detect, kernel_e2,iterations=4)
# g_to_detect_3 = cv2.erode(g_to_detect_2, kernel_e3,iterations=1)
# mostrar_imgs(['det2', 'det3'], [g_to_detect_2, g_to_detect_3])
mostrar_img('edges conv', g_to_detect_2)

In [ ]:
#Esqueletizar para operar solo con la esqueletización
#Se pueden borrar puntos esqueletizados más fácil
img_ezq = g_to_detect_2.copy()
size = np.size(img_ezq)
skel = np.zeros(img_ezq.shape,np.uint8)
element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
done = False
while( not done):
    eroded = cv2.erode(img_ezq,element)
    temp = cv2.dilate(eroded,element)
    temp = cv2.subtract(img_ezq,temp)
    skel = cv2.bitwise_or(skel,temp)
    img_ezq = eroded.copy()
 
    zeros = size - cv2.countNonZero(img_ezq)
    if zeros==size:
        done = True
mostrar_img('skeleton', skel)

In [ ]:
with_lines = original_td.copy()
lines = cv2.HoughLines(g_to_detect_2,10,np.pi/90,800)
third_1 = (int)(g_to_detect.shape[1]/3)
third_2 = (int)(g_to_detect.shape[1]*(2/3))
epsilon = (int)(0.01*g_to_detect.shape[1])
if type(lines) is np.ndarray:
    for line in lines:
        for rho,theta in line:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 2000*(-b))
            y1 = int(y0 + 2000*(a))
            x2 = int(x0 - 2000*(-b))
            y2 = int(y0 - 2000*(a))        
            if (x1 != x2 and y2 != y1):
                m = (y2-y1)/(x2-x1)
                b = y2 - m*x2
                x = (g_to_detect.shape[0]-b)/m
                if (x > 0 and x < g_to_detect.shape[1]):
                    if (x < third_1 and theta < 0.49 and theta > 0.2):
                        cv2.line(with_lines,(x1,y1),(x2,y2),(255),1)
                    if (x > third_2 and theta > 2.65 and theta < 2.92):
                        cv2.line(with_lines,(x1,y1),(x2,y2),(255),1)
                    if (x > third_1 and x < third_2 and (theta > 2.75 or theta < 0.39)):
                        cv2.line(with_lines,(x1,y1),(x2,y2),(255),1)

In [ ]:
mostrar_img('lines', with_lines)

In [ ]:
diff = cv2.subtract(with_lines, g_to_detect)
mostrar_img('diferencia', diff)

### Imagen de prueba para clasificación
Se segmenta y se calculan bordes para tener referencia visual (No lo requiere el algoritmo final)

In [ ]:
file_name = 'artificial_1'
original_td = cv2.imread(img_path + file_name + '.png', cv2.IMREAD_COLOR)
# original_td = h_crop(original_td, 0.1) #Para que queden de igual ancho
to_detect = cv2.cvtColor(original_td, cv2.COLOR_RGB2HSV)
to_detect = v_crop_top(segment_by_color(to_detect), top_crop_ratio)

In [ ]:
kernel = np.ones((3,3), np.uint8) 
to_detect = cv2.erode(to_detect, kernel, iterations=1)

In [ ]:
# mostrar_img('to_detect', to_detect)

In [ ]:
to_detect_edge = cv2.Canny(to_detect,50,150)
zeros = np.zeros(to_detect.shape, np.uint8)
three_channel_edge = cv2.merge((to_detect, zeros, zeros))
mostrar_img('edges canny', three_channel_edge)

### Clasificación (FIP + RCRD)
Trabajo en segmentos, divide la imagen en 20 segmentos (se puede optar por análizar solo la parte inferior, los primeros 15 segmentos)

In [ ]:
td_h, td_w = to_detect.shape
seg_height = (int)(td_h/segments_amount)
steps = (int)(td_h/seg_height) #Si td_h no es múltiplo de la cantidad de seg => steps != segments_amount
window_width = (int)(td_w*window_ratio)

#RCRD descripto como vectores para que sea más rápido verificar
offset_seg_height = 0
rcrd_vectors = np.zeros((steps, td_w), np.bool)
for i in range(0,steps):
    segment = seg_and[offset_seg_height:offset_seg_height + seg_height, 0:td_w]
    rcrd_vectors[i,:] = calculate_row(segment, seg_height, td_w)
    offset_seg_height = offset_seg_height + seg_height

In [ ]:
def is_crop(window):
    return True
    sum = 0
    for elem in window:
        if elem:
            sum = sum + 1
    sz = np.size(window)
    return ((sum/sz) > 0.80)
        
def verificate_with_rcrd(seg_index, window_pos, window_width):
#     return True
    return np.any(rcrd_vectors[seg_index, window_pos:window_pos + window_width])

In [ ]:
#FIP
offset_seg_height = 0

# from matplotlib import pyplot as plt
# x = np.arange(0,td_w) 
three_ch_copy = three_channel_edge.copy()

#La imagen la voy recorriendo de abajo hacia arriba
for seg_index in range(0, steps):
    #Calculo los vectores de cada segmento
    segment = to_detect[offset_seg_height:offset_seg_height + seg_height,0:td_w]
    vector = calculate_row(segment, seg_height, td_w)
    
    #Todo lo que no sea crop, es weed
    window_pos = 0;
    while (window_pos < td_w - window_width ):
        #Hay algo mal en el algoritmo
        if (is_crop(vector[window_pos:window_pos + window_width])):
            if (verificate_with_rcrd(seg_index, window_pos, window_width)):
            #Luego de verificar que sea crop, pinto ese sector
                sector = three_ch_copy[offset_seg_height:offset_seg_height + seg_height, window_pos: window_pos + window_width]
                sec_b, sec_g, sec_r = cv2.split(sector)
                sector = cv2.merge((sec_r, sec_g, sec_b)) #Lo pinto dando vuelta los canales
                three_ch_copy[offset_seg_height:offset_seg_height + seg_height, window_pos: window_pos + window_width] = sector
                window_pos = window_pos + window_width
            else:
                window_pos = window_pos + 1
        else:
            window_pos = window_pos + 1
    offset_seg_height = offset_seg_height + seg_height

In [ ]:
mostrar_img('th', three_ch_copy)

### Contagio de color

In [ ]:
#De manera recursiva repinta lo que encuentre a su alrededor
def paint(pixel):
    if (pixel[0]):
        aux = pixel[0]
        pixel [0] = pixel[2]
        pixel[2] = aux

def contagio(img, y_init, x_init):
    img_h, img_w, ch = img.shape
    x1= x_init-1
    x2= x_init+1
    y1= y_init-1
    y2= y_init+1
    if (x1 >= 0): paint(img[y_init,x1])
    if (x2 < img_w): paint(img[y_init,x2])
    if (y1 >= 0): paint(img[y1,x_init])
    if (y2 < img_h): paint(img[y2,x_init])

In [ ]:
three_ch_copy2 = three_ch_copy.copy()
for y in range(0,td_h-1):
    for x in range(0,td_w-1):
        if (three_ch_copy2[y,x,2]): contagio(three_ch_copy2, y, x) #BGR, el contagio es con R
mostrar_imgs(['th contagio','th'], [three_ch_copy2, three_ch_copy])

In [ ]:
#Colorearlo en imagen de bordes (Poner valor 255 en segundo (crop) o tercer canal (weed)) 
#Sumar bordes clasificados con imagen 
result = cv2.add(original_td, three_ch_copy2)

In [ ]:
mostrar_img('resultado', result)

### Guardado de imágenes

In [ ]:
#Guardar resultado
file_name='total'
cv2.imwrite('img/row_test/' + file_name + '_resultado2.png', result)
# cv2.imwrite('img/row_test/' + file_name + '_with_lines.png', with_lines)